<a href="https://colab.research.google.com/github/ZoeyZoZo119/Hello_World/blob/master/758B_Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Load data**
1. Using tradintional ML-random forest to make prediction-(Unigram- TF/IDF)
2. Fully-connected forward network w/t TF/IDF





In [84]:
# load text data and convert the label/sentiment into corresponding numeric values: ' positive' :2, 'neutral':1, 'negative':0
# possible package you might need are: panda, numpy
import pandas as pd
import numpy as np





## PART 1- Traditional ML

In [85]:
# read the training data
fname = 'facebook_comments.csv'
df_train = pd.read_csv(fname,header=None, names=['text','sentiment'],encoding='iso-8859-1',lineterminator='\n')
  #* encoding-->to ensure ml can be exerted in any os
  #* create one more column(label) and map sentiment categories into numeric values : ' positive' :2, 'neutral':1, 'negative':0 
sent ={'positive' :2, 'neutral':1, 'negative':0}
df_train['labels'] =df_train['sentiment'].str.strip().map(sent)
  #** extract one columns = series--> use (str.strip()) to remove heading and spaces for every elements--> (map(sent)) replace every element in 
  #** that serise to corresponding value

# get texts and labels
  #* extract 1st column(text) and 3rd column(labels) as X and Y and convert them into array
training_texts = df_train.text.values
labels = df_train.labels.values

print(type(training_texts), type(labels))

# show the first 5 records
df_train.head()

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


,text,sentiment,labels
0,Heres a single to add to Kindle. Just read t...,neutral,1
1,If you tire of Non-Fiction.. Check out http://...,neutral,1
2,Ghost of Round Island is supposedly nonfiction.,neutral,1
3,Why is Barnes and Nobles version of the Kindle...,negative,0
4,@Maria: Do you mean the Nook? Be careful bo...,positive,2


# Preprocess data



In [86]:
# preprocess the loaded textual data, including stepwords, stemming, and tokenization, etc.
# represent each docu ent (i.e. comment) using TF-IDF strategy. The features are the top frequent unigrams across all comments.
# possible package you might need are: scikit-learn, numpy
from sklearn.feature_extraction.text import TfidfVectorizer

# tokenize and create a document-features matrix X and a label vector Y
vectorizer = TfidfVectorizer(stop_words = 'english', max_features = 500, ngram_range = (1,1))
  #*ngram_range(1,1)unigram; ngram_range(1,2)combination of unigram and bigram. (2,2) bigram only.
  #*create TF/IDF objects; setup top frequent 500 unigrams as features 

# the matrix containt instance of TF/IDF is sparse matrix, not nparray, so they need to be converted into nparray --> apply TF/IDF objects to the data being transformed.
instance = vectorizer.fit_transform(training_texts) # get sparse matrix
X = instance. toarray() # convert sparse matrix to nparray
Y = labels

# print out the shape of X and Y
print(X.shape, ',', Y.shape)
  #* X is a 1999 * 500 matrixs; Y is a 1999 rows vector.
print(Y[:10])
print(X[0,:50])

(1999, 500) , (1999,)
[1 1 1 0 2 2 2 0 2 0]
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.28915636 0.         0.         0.
 0.         0.         0.2971592  0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]


#  TAKE AWAY(Preprocess Data): Build up a nparray matrix for X :
>There are 1999 documents. We set words in each doc. as unigram, and extract 500 most frequent unigrams in each doc by using TF/IDF function. Then, we set 1999 doc. as rows and 500 features as columns to create a 1999*500 matrix. Every data that represent one feature in one doc. is called one instance. This matrix is called sparse matrix(稀疏数列）, because most of instance are 0. Since for each doc.,some features are not included.  `They want to save the storage???` 

>We are going to use Tensor. We want to get Tensor from nparray. 

## Algorithm: Traditional ML Models- Random Forest

In [87]:
# using 10-fold cross-validation to show the prediciton accuracy
#-->9 folds = training set, rest of fold = validation; repeat 10 times to get average performance
# possible package needed are: scikit-learn, numpy
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier

kfold = KFold(n_splits=10, shuffle=True, random_state=2020) 
  #** shuffle--randomly iterate 10 folds in 10 times
  #** random-state--random list is controled by random generator inside of computer, if to fix the random_state the random generator is fixed. 
# Then run the same code in the same computer will get the same result. But way the state is numerical value??
rf_model = RandomForestClassifier(criterion='entropy', max_depth=2, random_state=2020)

# Train model 
rf_cvscores= []
# using for loop make 10 times iteration in Training dataset which is splited into 10 fold; 9 for train model, 1 for validation model. 
for train_idx, val_idx in kfold.split(X):  #using index to get correponding rows
  rf_model.fit(X[train_idx], Y[train_idx]) # train model 
  acc = rf_model.score(X[val_idx],Y[val_idx]) # validate model 
  rf_cvscores.append(acc)


print('Random Forest - mean: %.4f%% (std: +/- %.4f%%)'%(np.mean(rf_cvscores)*100, np.std(rf_cvscores)*100))

Random Forest - mean: 64.1332% (std: +/- 2.0919%)


## Fully-connected feedforward Neural Network

Degisn a network: 
1. Input and output: 500 features--> 500 inputs --> 2 hidden layers-->3 output: positive/neutral/ negative
2. The number of neurons in each hidden layers
3. algorithem of each neuron --> relu/ the dropout of each hidden layer
4. algorithem of output --> softmax







In [88]:
# Design your own network with the following requirement:
# 1. Having dropout
# 2. Separate the dataset into training and validation (80-20%)
# 3. The prediciton accuracy on the validation set should be at least 50% for this 3-class classification problem 

# possible package you might need are: scikit-learn, numpy, torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

Build the train loader and validation loader

In [89]:
# convert numpy array to TensorDataset and create a data loader for training and validation, respectively
# some hyperparameters: input dimension, output dimension, batch size, number of epochs, and the learning rate.

epochs = 5
lr = 1e-4
indim = X.shape [1] # X= 1999*500 matrix-->X.shap[1]=for each instance it has 500 columns
outdim = 3 
drate = 0.7 # dropout rate
batch_size = 16



In [90]:
# Build the network
# create a train loader and validation loader 
# conver x,y numpy array to tensor in to one dataset(x,y)--> train set/ test set--> when you call the loader what data should be included in each batch
# e.g. to see the text and labels in one batch 
# Divide the tensor dataset( x_tensor, y_tensor)again into training set and validation set--> create a train_loader/val_loader
X_tensor = torch.from_numpy (X)
Y_tensor = torch.from_numpy (Y)

dataset = TensorDataset(X_tensor, Y_tensor) 
train_size = int(0.8 *len(dataset))
val_size = len(dataset) - train_size
# get training dataset 
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size, shuffle = True)
val_loader = DataLoader(val_dataset,batch_size, shuffle = True)


In [107]:
# Buildup a fully-connected forward network structure 
class SentimentNetwork(nn.Module):
  def __init__(self,input_dim,output_dim,dropout_rate):
    super(SentimentNetwork,self).__init__()
    self.fe1 = nn.Linear(input_dim,100)
    self.fe2 = nn.Linear(100,50)
    self.fe3 = nn.Linear(50,output_dim)
    self.do1 = nn.Dropout(dropout_rate)
    self.do2 = nn.Dropout(dropout_rate)

    def forward(self,x):
      x= F.Relu(self.fe1(x))
      x=self.do1(x)
      x=F.Relu(self.fe2(x))
      x=self.do2(x)
      x=F.log_softmax(self.fe3(x))
      return x

model = SentimentNetwork(indim,outdim,drate)
print(model)


SentimentNetwork(
  (fe1): Linear(in_features=500, out_features=100, bias=True)
  (fe2): Linear(in_features=100, out_features=50, bias=True)
  (fe3): Linear(in_features=50, out_features=3, bias=True)
  (do1): Dropout(p=0.7, inplace=False)
  (do2): Dropout(p=0.7, inplace=False)
)


## Create a training function to train the model and an evalutation function to evaluate the performance on the separate validation set.

In [108]:
# def a training process function for one epoch
def train(model, train_loader, optimizer, criterion):
  train_loss,train_acc = 0.0, 0.0 # the loss and accuracy for each epoch
 
  model.train()
  for batch_x, batch_y in train_loader:
    optimizer.zero_grad()   
    # 0 graident
    predcitions = model(batch_x)
    # predcitions = get the predicted output for the current batch batch_x；forward pass: feed inputs to the model to get outputs 
    predcitions = predictions.detach().numpy()
    #convert torch variable to numpy: predictions. detach().numpy()
    loss = criterion(predcitions, batch_y)  
    # loss - cal the loss using predicted output and the truth for the current bathc: predictions, batch_y
    predicted = torch.argmax(predcitions, 1)
    acc += (predicted == batch_y).sum().item()
    loss.backward() 
    # backward: perform gradient descent of the loss w.r. to the model params
    optimizer.step()  
    # update the model parameters by performing a single optimization step
    
    # accumulate the training loss
    epoch_loss += loss.item()
    # accumulate the training accuracy
    epoch_acc += acc
    # cal the average epoch_loss and epoch_acc
    epoch_loss /= len(train_loader)
    epoch_acc /= len(train_loader)

   
    return epoch_loss, epoch_acc





In [109]:
# define a validation/evaluation process function 
def evaluate(model, val_loader, criterion):
 val_lose, val_acc = 0.0, 0.0 # the loss and accuracy for each epoch 
 model.eval()
 with torch.no_grad():
   for batch_x, batch_y in val_loader:
      predcitions = model(batch_x)
      # predcitions = get the predicted output for the current batch batch_x；forward pass: feed inputs to the model to get outputs 
      predcitions = predictions.detach().numpy()
      #convert torch variable to numpy: predictions. detach().numpy()
      loss = criterion(predcitions, batch_y) 
      # acc
      predicted = torch.argmax(predcitions, 1)
      acc += (predicted == batch_y).sum().item()
      
      epoch_loss +=loss.item()
      
      epoch_acc += acc
      return val_loss, val_acc
# calculate the average epoch_loss and epoch_acc
      epoch_loss /= len(val_loader)
      epoch_acc /= len(val_loader)

      

In [111]:
# define the loss function and optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)
# if MSELoss = (predictions, truth): batch_size * output_dim; truth: batch_size * output_dim

#real training and evaluation process
for epoch in range (epochs):   
  train_loss, train_acc = train(model, train_loader, optimizer, criterion)
  valid_loss, vaild_acc = evaluate(model, val_loader, criterion)

  print(f'Epoch:{epoch +1:0.2}')
  print(f'\tTrain Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}')
  print(f'\t Val.Loss: {valid_loss:.4f} | Val.Acc: {valid-acc:.4f}')

NotImplementedError: ignored